https://docs.ragas.io/en/stable/getstarted/evaluation.html

In [1]:
import json
import os
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

from datasets import Dataset

In [2]:
import pandas as pd
df = pd.read_excel("verify_result.xlsx")
df.head(3)

,name,category,title,question,stance,reference,answer,reasoning,evidence,source_content,source_category,source_sub_category,cost
0,Joe Biden,Social Issues,Abortion,Do you support women's right to abortion?,Yes,NaN,yes,Joe Biden has expressed clear support for a wo...,['I would work to enshrine into federal law th...,I would work to enshrine into federal law the ...,"Abortion, Pro-Life & Genetic Engineering",Roe v. Wade,0
1,Joe Biden,Social Issues,Gender transition,Should people under the age of 18 years old be...,Yes,NaN,yes,Joe Biden has expressed support for transgende...,"[""The fact of the matter is that we're in a po...",would be more expensive to treat if they had t...,Health & Medical,Medically Uninsured or Underinsured,0
2,Joe Biden,Social Issues,Hate Speech,Should hate speech be protected by the first a...,No,NaN,unknown,The provided context does not contain a direct...,[],[4:51] Former Vice President Joe Biden discuss...,"Crime, Police & Imprisonment",Hate Crimes,0


In [3]:
from genie_master import GenieMaster

genie_db_path = "./chroma_qadata_db/"
gm = GenieMaster(db_path=genie_db_path)
print("Genie is ready...", gm.model_is_ready())
print("Document collection count...", gm._document_count())

Genie Master initialized at: 2024-01-20 04:18:11.464888-05:00
Genie is ready... True
Document collection count... 45990


In [4]:
name = "Joe Biden"
genie = gm.get_genie(name)
print(genie)

<Genie name=Joe Biden model=gpt-3.5-turbo-1106>


In [5]:
df_sample = df.sample(5)
questions = df_sample.question.to_list()
questions

['Should there be more restrictions on the current process of purchasing a gun?',
 'Should political candidates be required to release their recent tax returns to the public?',
 'Should working illegal immigrants be given temporary amnesty?',
 'Should "gender identity" be added to anti-discrimination laws?',
 'Should the US increase foreign aid spending?']

# new

In [7]:
eval = genie.evaluate(questions)
eval

evaluating with [context_precision]


100%|██████████| 1/1 [00:18<00:00, 18.97s/it]


evaluating with [faithfulness]


100%|██████████| 1/1 [00:17<00:00, 17.10s/it]


evaluating with [answer_relevancy]


100%|██████████| 1/1 [00:05<00:00,  5.57s/it]


{'context_precision': 0.0000, 'faithfulness': 0.7000, 'answer_relevancy': 0.7096}

In [9]:
df_eval = eval.to_pandas()
df_eval.head()

,question,answer,contexts,context_precision,faithfulness,answer_relevancy
0,Should there be more restrictions on the curre...,"{""answer"": ""yes"", ""reasoning"": ""Joe Biden has ...",[Joe Biden helped author the Federal Assault W...,0.0,1.0,0.897981
1,Should political candidates be required to rel...,"{""answer"": ""yes"", ""reasoning"": ""Joe Biden has ...","[Wealthy special interests, corporations, and ...",0.0,0.5,0.919536
2,Should working illegal immigrants be given tem...,"{""answer"": ""yes"", ""reasoning"": ""Joe Biden supp...",[The millions of undocumented people in the Un...,0.0,1.0,0.842690
3,"Should ""gender identity"" be added to anti-disc...","{""answer"": ""yes"", ""reasoning"": ""Joe Biden has ...",[While workplace discrimination is not as obvi...,0.0,1.0,0.887627
4,Should the US increase foreign aid spending?,"{""answer"": ""unknown"", ""reasoning"": ""The quotes...","[funding increase, as recommended by several v...",0.0,0.0,0.000000


# old

In [ ]:
base_batch_result = genie.base_batch_ask(questions)
base_batch_result

In [ ]:
df_result = pd.DataFrame(base_batch_result)
df_result["answer"] = df_result["result"].apply(lambda x: json.dumps(x))
df_result["contexts"] = df_result["context"].apply(lambda docs: [doc.page_content for doc in docs])
df_result = df_result[["question", "answer", "contexts"]]
df_result

In [ ]:
dataset = Dataset.from_pandas(df_result)
dataset

In [ ]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

In [ ]:
from ragas import evaluate

result = evaluate(
    dataset.select(range(2)),
    metrics = [
        context_precision,
        faithfulness,
        answer_relevancy,
        # context_recall
    ]
)
result